In [2]:
import requests, zipfile,io
import urllib.request
#http framework to make Mapbox Matric API requests for routes
import json # handle response as json
import datetime # save timestamp
import rasterio
import numpy as np
import time
import geopandas as gpd
import pandas as pd
import os.path
from shapely.geometry import Point
from shapely.geometry import box
from utils import *
from closest_hospital import *
from origin_destination import mapbox_matrix_API

## Paramètres et création des dossiers

In [3]:
MAPBOX_ACCESS_TOKEN = 'pk.eyJ1IjoibWlrbWVoMDEiLCJhIjoiY2s4YzE3ZnB3MGhoNDNsdHEzNG41N2VpNCJ9.mmNr9093SAZnZwULlKY2cQ'

country = 'Tunisia'
# Country code ISO3

country_csv = pd.read_csv('wbccodes2014.csv')
country_ISO = country_csv.set_index('country_name').loc[country,'country']
output_path = os.path.join(country_ISO,'output')

if not os.path.exists(country_ISO):
    os.makedirs(country_ISO)
    
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
pop_folder = os.path.join(country_ISO,'Data Pop')
location = 'WorldPop'
print("Getting %s population data"%(location) )

if not os.path.exists(pop_folder):
    os.makedirs(pop_folder)

map_file = os.path.join(pop_folder, '%s_ppp_2019.tif'%country)

# Downloading the raster file for population unless it's already downloaded
if not os.path.isfile(map_file):
    
    print('Beginning file download with urllib2...')
    url = 'ftp://ftp.worldpop.org.uk/GIS/Population/Global_2000_2020/2019/%s/%s_ppp_2019.tif' %(country_ISO,country_ISO)
    urllib.request.urlretrieve(url, map_file)


world_shp_path = 'World_shp'
if not os.path.exists(world_shp_path):
    os.makedirs(world_shp_path)
    
    # Downloading and extracting file
    url_world_shp = 'https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/cultural/ne_50m_admin_0_countries.zip'
    r = requests.get(url_world_shp, stream=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(world_shp_path)
    
world_shp = gpd.read_file(os.path.join(world_shp_path,'ne_50m_admin_0_countries.shp'))
country_shp = world_shp[world_shp.ADM0_A3==country_ISO.upper()]

country_centroid = country_shp.unary_union.centroid
lat,lon = country_centroid.bounds[1],country_centroid.bounds[0]
        
# formula below based on :https://gis.stackexchange.com/a/190209/80697
epsg = int(32700-np.round((45+lat)/90,0)*100+np.round((183+lon)/6,0))


Getting WorldPop population data
Beginning file download with urllib2...


## Creating the points from where we will seek for the closest hospital

In [4]:
# Browsing the raster file, cutting it into squares of size #window
# and computing bounds, population and number of null values

origins = pd.DataFrame()
window = 500
with rasterio.open(map_file) as src:
    for left_x in np.arange(0,src.width,window):
        for top_y in np.arange(0,src.height,window):
            out = get_pop(map_file,left_x,top_y,window,plot=False)
            if out != {}:
                origins = origins.append([out])
        print("%i/%i\r"%(left_x,src.width),end="")

**Important : Run many times the cell below**

In [7]:
# Splitting the original big squares into smaller squares.
# You can run this cell as many times as you want depending 
# on the precisions you are seeking. Hence, running it multiple
# times will ask for more computational power later.

print("%i regions need splitting"%len(origins[origins['split']==True]))
olen = len(origins)
for i in np.arange(olen):
    print("%i/%i\r"%(i+1,olen),end="")
    if origins.iloc[i,origins.columns.get_loc('split')] == True:
        origins.iloc[i,origins.columns.get_loc('split')] = 'done'
        s = split(map_file,origins.iloc[i])
        origins = origins.append(s,sort=False)
print("done.")
print("We now have %i regions of min size %i, %i will be split in next round"%\
      (len(origins),origins['window'].min(),len(origins[origins['split']==True])))

1564 regions need splitting
done.2392
We now have 16450 regions of min size 62, 13166 will be split in next round


In [8]:
origins = origins[origins['tot_pop']>0]
origins = origins[origins['split']!='done']
print("We have %i regions of size %i, %i with population >0"%
      (len(origins),min(origins['window']),len(origins[origins['tot_pop']>0])))

#Transform it to geopandas frame where each row corresponds 
#to a location from where we will compute travel time to closest hospital
origins = gpd.GeoDataFrame(origins,crs='epsg:4326', geometry=[Point(xy) for xy \
                                                              in zip(origins['center_lon'], origins['center_lat'])])

We have 13166 regions of size 62, 13166 with population >0


## Hospitals

In [11]:
hospitals_path = os.path.join(country_ISO, 'Hospitals')
if not os.path.exists(hospitals_path):
    os.makedirs(hospitals_path)

hospital_shp_path = os.path.join(hospitals_path, '%s.shp'%country)
if not os.path.exists(hospital_shp_path):
    # Downloading and extracting file
    url_hospital = 'https://healthsites.io/data/shapefiles/%s.zip'%country
    r = requests.get(url_hospital, stream=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(hospitals_path)



In [12]:
hospital_shp_path = os.path.join(hospitals_path, '%s.shp'%country)
hospitals = gpd.read_file(hospital_shp_path)

hospitals = hospitals[(hospitals.healthcare.isin(['clinic','hospital'])) | (hospitals.amenity.isin(['clinic','hospital'])) ]
print('Number of hospitals/clinics :', len(hospitals))

Number of hospitals/clinics : 318


## Computing time travel

In [13]:
origins = origins.reset_index(drop=True)
origins_crs = convert_crs_gdf(origins,4326,epsg)
hospitals_crs = hospitals.to_crs(epsg=epsg)

In [15]:
ori = origins_crs.copy()
o_type = 'hospital'
cols=['t_'+o_type,'m_'+o_type,'so_'+o_type]
for col in cols:
    ori[col]=-1

In [16]:
output = mapbox_matrix_API(ori,hospitals_crs,MAPBOX_ACCESS_TOKEN,epsg, name='hospital',n_keep=2)

Doing 13166


C:\Users\mehdi\Desktop\COVID\utils.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  gdf["geometry"] = gdf["geometry"].apply(lambda geom: shapely.wkb.loads(geom))


Doing batch 1098, from 13164 to 13176, of 13166

returning


### Exportation des données

In [17]:
origins['t_hospital'] = output['t_hospital']
origins['so_hospital'] = output['so_hospital']

# on garde que les points à l'intérieur du pays
gpd.sjoin(origins,country_shp,op='within')[origins.columns].to_file(os.path.join(output_path, 'origins.shp'))
hospitals.to_file(os.path.join(output_path, 'hospitals.shp'))

C:\Users\mehdi\Anaconda3\envs\GDAL\lib\site-packages\geopandas\tools\sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
C:\Users\mehdi\Anaconda3\envs\GDAL\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
